# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
# Store filepath in a variable
data_load = "Download_weather_data.csv"

# Read our Data file with the pandas library
data_load_df = pd.read_csv(data_load, encoding="ISO-8859-1")

# Show just the header
data_load_df.head()

,Unnamed: 0,city,Cloudiness,Humidity,Wind Speed,Temp Max,Date,Country,Latitude,Longitude
0,0,rikitea,55,73,7.38,77.45,1587507746,PF,-23.12,-134.97
1,1,punta arenas,89,53,44.96,50.00,1587507505,CL,-53.15,-70.92
2,2,sur,89,39,10.45,87.28,1587507680,OM,22.57,59.53
3,3,alofi,75,66,13.87,84.20,1587507747,NU,-19.06,-169.92
4,4,weiden,1,43,8.05,50.00,1587507747,DE,49.68,12.16


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
# Store latitude and longitude in locations
cities = data_load_df[["Latitude", "Longitude"]]


In [22]:
#Create Variable
Humidity = data_load_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(cities, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Create new DataFrame
narrow_down_df = data_load_df.loc[((data_load_df["Temp Max"]< 80) & (data_load_df["Temp Max"]>70)) & (data_load_df["Cloudiness"] ==0) &(data_load_df["Wind Speed"]<10)]
narrow_down_df = narrow_down_df.dropna(how='any')
narrow_down_df.reset_index(inplace=True)
narrow_down_df = pd.DataFrame(narrow_down_df , columns = ["city","Country","Cloudiness","Latitude", "Longitude", "Temp Max","Wind Speed"])
narrow_down_df.head()

,city,Country,Cloudiness,Latitude,Longitude,Temp Max,Wind Speed
0,kununurra,AU,0,-15.77,128.73,73.40,1.81
1,cidreira,BR,0,-30.18,-50.21,70.65,5.64
2,awjilah,LY,0,29.11,21.29,75.09,9.44
3,inhambane,MZ,0,-23.86,35.38,73.60,9.33
4,oussouye,SN,0,12.48,-16.55,78.80,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_list = []

# Create DataFrame to add hotel name column
hotel_df =pd.DataFrame(narrow_down_df, columns=["city", "Country", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df                                     


,city,Country,Latitude,Longitude,Hotel Name
0,kununurra,AU,-15.77,128.73,
1,cidreira,BR,-30.18,-50.21,
2,awjilah,LY,29.11,21.29,
3,inhambane,MZ,-23.86,35.38,
4,oussouye,SN,12.48,-16.55,
5,santa maria,BR,-29.68,-53.81,
6,natalio,PY,-26.72,-55.08,
7,waddan,LY,29.16,16.14,
8,jalu,LY,29.03,21.55,
9,dera ghazi khan,PK,30.06,70.63,


In [13]:
# Set parameters
params = {
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
    }

# Set location
for index, row in hotel_df.iterrows():
    # get latitude, longitude from dataframe
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    params["location"]=f"{lat},{lng}"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
        # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print("Missing field/result... skipping.")
        
#Display DataFrame        
hotel_df

Missing field/result... skipping.


,city,Country,Latitude,Longitude,Hotel Name
0,cidreira,BR,-30.18,-50.21,Novomar Pousada e Turismo Ltda
1,castro,BR,-24.79,-50.01,CHACARA BAILLY
2,chuy,UY,-33.70,-53.46,Nuevo Hotel Plaza
3,khakhea,BW,-24.69,23.49,
4,salta,AR,-24.79,-65.41,Hotel Alejandro I
5,pyapon,MM,16.28,95.68,Aung Naing Thu Motel
6,balkanabat,TM,39.51,54.37,Nebitchi Hotel
7,sarakhs,IR,36.54,61.16,Sarakhs Abrisham Hotel


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [26]:
# Add marker layer ontop of heat map
marker_layer =gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add layer
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))